In [7]:
import os
import numpy as np
import cv2
import skimage.restoration as restoration
import pywt
import skimage.metrics as metrics
import matplotlib.pyplot as plt
import random
import pandas as pd


In [9]:
# Function to load images
def load_images(image_dir):
    images = []
    for filename in os.listdir(image_dir):
        if filename.endswith(".jpeg"):
            img = cv2.imread(os.path.join(image_dir, filename), cv2.IMREAD_GRAYSCALE)
            images.append(img)
    return images

# Preprocessing function
def preprocess_images(images, historgram_eq=True):
    processed_images = []
    for img in images:
        # Convert to grayscale if not already (assuming image is loaded in grayscale)
        # Resize if necessary (skipped here)
        # Normalize the intensity values
        img_normalized = cv2.normalize(img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        # Histogram equalization
        if historgram_eq:
            img_eq = cv2.equalizeHist(np.uint8(img_normalized*255))
        processed_images.append(img_eq)
    return processed_images

# Noise reduction functions
def apply_nlm(images, h=1.0):
    nlm_images = [restoration.denoise_nl_means(img, h=h, fast_mode=True) for img in images]
    return nlm_images

def apply_wavelet(images):
    wavelet_images = []
    for img in images:
        coeffs2 = pywt.dwt2(img, 'bior1.3')
        LL, (LH, HL, HH) = coeffs2
        wavelet_images.append(LL)
    return wavelet_images

# Performance evaluation function
def evaluate_images(original, processed):
    psnr_values = [metrics.peak_signal_noise_ratio(original[i], processed[i]) for i in range(len(original))]
    return psnr_values

# Visualization function
def visualize_comparisons(original, nlm, wavelet):
    fig, axs = plt.subplots(nrows=len(original), ncols=3, figsize=(15, 5*len(original)))
    for i in range(len(original)):
        axs[i, 0].imshow(original[i], cmap='gray')
        axs[i, 0].set_title('Original')
        axs[i, 1].imshow(nlm[i], cmap='gray')
        axs[i, 1].set_title('NLM')
        axs[i, 2].imshow(wavelet[i], cmap='gray')
        axs[i, 2].set_title('Wavelet')
        for ax in axs[i, :]:
            ax.axis('off')
    plt.show()


In [11]:
# Pipeline execution
image_dir_normal = 'images/chest_xray/test/NORMAL'
image_dir_pneumonia = 'images/chest_xray/test/PNEUMONIA'  
original_images = load_images(image_dir_normal) + load_images(image_dir_pneumonia)
random.shuffle(original_images)

In [12]:
preprocessed_images = preprocess_images(original_images)
nlm_images = apply_nlm(preprocessed_images)
wavelet_images = apply_wavelet(preprocessed_images)
psnr_nlm = evaluate_images(original_images, nlm_images)
psnr_wavelet = evaluate_images(original_images, wavelet_images)
visualize_comparisons(original_images, nlm_images, wavelet_images)

KeyboardInterrupt: 